In [1]:
pip install chromedriver-autoinstaller


C:\Users\User\Web_Crawling\nerdy>doskey defects4j=perl C:\Users\User\defects4j\framework\bin\defects4j $* 
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install selenium


C:\Users\User\Web_Crawling\nerdy>doskey defects4j=perl C:\Users\User\defects4j\framework\bin\defects4j $* 
Note: you may need to restart the kernel to use updated packages.


In [3]:
import chromedriver_autoinstaller
from selenium import webdriver
import time

class Driver:
    @staticmethod
    def driver_init():
        chromedriver_autoinstaller.install()
        options = webdriver.ChromeOptions()
#         options.add_argument('headless') # jupyter notebook 환경에서 주석처리
        options.add_argument('lang=ko_KR')
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36")
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-notifications')
        options.add_argument('--disable-popup-blocking')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-browser-side-navigation')
        options.add_argument('disable-gpu')

        driver = webdriver.Chrome(options=options)
        driver.set_window_size(1600, 1100)

        return driver

In [4]:
driver = Driver.driver_init()

In [5]:
def find_product_paths(urls):
    product_paths = []
    
    ## 상의/자켓/하의 분류 루프
    for (url, pages_cnt) in urls:
        ## 각 옷 분류의 페이지 갯수 루프
        for page_num in range(pages_cnt):
            try:
                driver.get(url + "&page=" + str(page_num+1))
            except:
                continue

            products = driver.find_elements('xpath', '//ul[@class="prdNeo prd_set"]/li')
            products_cnt = len(products)

            ## 각 product의 path를 product_paths 리스트에 저장
            for i in range(products_cnt):
                product = driver.find_elements('xpath', '//ul[@class="prdNeo prd_set"]/li')[i]
                product_path = product.find_element('xpath', './div/div[@class="thumbnail on"]/a').get_attribute('href')
                product_paths.append(product_path)
                
    return product_paths

In [6]:
def find_all_color_paths(product_paths):
    all_color_paths = []
    
    for product_path in product_paths:
        ## 기존의 product_path 추가
        all_color_paths.append(product_path)
        
        try:
            driver.get(product_path)
        except:
            continue
            
        other_color_products = driver.find_elements('xpath','//div[@id="product_relation"]/div[@class="xans-element- xans-product xans-product-relationlist relate_thumbs_img noSlide"]/ul/li')
        other_products_cnt = len(other_color_products)
        
        ## 다른 색들의 product_path 추가
        for i in range(other_products_cnt):
            other_product = driver.find_elements('xpath','//div[@id="product_relation"]/div[@class="xans-element- xans-product xans-product-relationlist relate_thumbs_img noSlide"]/ul/li')[i]
            other_product_path = other_product.find_element('xpath', './div/a').get_attribute('href')
            all_color_paths.append(other_product_path)
    
    return all_color_paths
            

In [7]:
def append_data(base_dict, cat, base_name, base_code, baseURL, color_name, base_gender, price, base_price, color_fabric, imageURL, size, is_available):
    base_dict["baseCat"].append(cat)
    base_dict["baseName"].append(base_name)
    base_dict["baseCode"].append(base_code)
    base_dict["baseUrl"].append(baseURL)
    base_dict["colorName"].append(color_name)
    base_dict["baseGender"].append(base_gender)
    base_dict["price"].append(price)
    base_dict["basePrice"].append(base_price)
    base_dict["colorFabric"].append(color_fabric)
    base_dict["imageUrl"].append(imageURL)
    base_dict["size"].append(size)
    base_dict["isAvailable"].append(is_available)

In [31]:
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException



def crawl_pages(product_paths):
    
    base_dict= {"baseCat":[], "baseName": [], "baseCode": [], "baseUrl": [], "colorName": [],
            "baseGender": [], "price": [], "basePrice": [], "colorFabric": [], "imageUrl": [],
            "size": [], "isAvailable": []
            }
    
    ## 드라이버 초기화
    driver = Driver.driver_init()
    ## 각 product에 대한 정보 추출
    for product_path in product_paths:
        driver.get(product_path)
        
        
        ## URL을 통해 cat 정보 추출
        cat = 'top'
        if "cate_no=30" in product_path or "cate_no=61" in product_path:
            cat = 'top'
        elif "cate_no=31" in product_path:
            cat = 'bottom'

        baseURL = product_path
        
        ## text_info element
        text_info = driver.find_element('xpath', '//div[@class="detail_top_right"]')
        
        
        ## 이미지 추출
        imageURL = []
        images = []
        images_cnt = 0
        if len(driver.find_elements('xpath','//div[@id="product_detail"]/div[@class="detailEditor"]/img')) != 0:
            images = driver.find_elements('xpath','//div[@id="product_detail"]/div[@class="detailEditor"]/img')
            images_cnt = len(images)
            for i in range(images_cnt):
                imgsrc = driver.find_elements('xpath','//div[@id="product_detail"]/div[@class="detailEditor"]/img')[i].get_attribute('src')
                imageURL.append(imgsrc)
        elif len(driver.find_elements('xpath','//div[@id="product_detail"]/div[@class="detailEditor"]/div[1]/img')) != 0:
            images = driver.find_elements('xpath','//div[@id="product_detail"]/div[@class="detailEditor"]/div[1]/img')
            images_cnt = len(images)
            for i in range(images_cnt):
                imgsrc = driver.find_elements('xpath','//div[@id="product_detail"]/div[@class="detailEditor"]/div[1]/img')[i].get_attribute('src')
                imageURL.append(imgsrc)
        else:
            continue
        
        ## base_name 추출
        base_name = text_info.find_element('xpath', './div[@class="prd_name_wrap "]/h2').text
        
        ## base_gender 추출
        base_gender = 0
        if "우먼스" in base_name:
            base_gender = 2
        
        ## 가격 추출
        price = ""
        base_price = ""
        
        trs = text_info.find_elements("xpath",'//td[@colspan="2"]/div') #td의 div 항목 갯수
        
        ## 할인을 하지 않는 경우
        if len(trs) == 2:
            price = text_info.find_element('xpath', '//div[@class="display_price"]/span[@id="sale_price"]').text
            base_price = price
            
        ## 할인을 하는 경우
        elif len(trs) == 3:
            price = text_info.find_element('xpath', '//div[@class="display_price"]/span[@id="sale_price"]').text
            base_price = text_info.find_element('xpath', '//div[@id="origin_price"]').text
        else:
            continue


        ## color_fabric 추출
        if len(driver.find_elements('xpath','//div[@class="addService"]/div[2]/div[2]/ul/li[2]')) != 0:
            color_fabric = driver.find_element('xpath','//div[@class="addService"]/div[2]/div[2]/ul/li[2]').get_attribute('textContent')
        elif len(driver.find_elements('xpath','//div[@class="addService"]/div[1]/div[3]/div[2]/ul/li[2]')) != 0:
            color_fabric = driver.find_element('xpath','//div[@class="addService"]/div[1]/div[3]/div[2]/ul/li[2]').get_attribute('textContent')
        else:
            continue
        
        
        select = ""
        color_name = ""
        base_code = ""
        
        ## option_title이 '색상'으로 되어 있는 경우
        if len(text_info.find_elements('xpath', '//select[@option_title="색상"]')) != 0:
            
            ## base_name에 옷의 색 정보가 포함되어있는 경우
            if len(text_info.find_elements('xpath', '//select[@option_title="색상"]/option')) == 3:
                         
                color_name = base_name.split(" ")[-1]
                base_code = base_name.replace(color_name, "")[:-1]
            
                select = Select(text_info.find_element('xpath', '//select[@option_title="색상"]'))
                try:
                    select.select_by_visible_text(color_name)
                except NoSuchElementException:
                    try:
                        color_text = color_name + " [품절]"
                        select.select_by_visible_text(color_text)
                    except:
                        continue
                except ElementNotInteractableException:
                    continue
                    
                size_elements = text_info.find_elements('xpath', '//select[@option_title="사이즈"]/option')[2:]
                size = "M"
                is_available = True

                for element in size_elements:
                    size = element.text.replace(" [품절]","")
                    if "[품절]" in element.text:
                        is_available = False
                    else:
                        is_available = True
                        
                append_data(base_dict, cat, base_name, base_code, baseURL, color_name, base_gender, price, base_price, color_fabric, imageURL, size, is_available)
            
            ## base_name에 옷의 색 정보가 포함되어 있지 않은 경우
            else:
                option_cnt = len(text_info.find_elements('xpath', '//select[@option_title="색상"]/option'))
                
                ## 각 색깔 별로 따로 정보를 추출 및 저장해야 함
                for i in range(2,option_cnt):
                    color_value = text_info.find_elements('xpath', '//select[@option_title="색상"]/option')[i].get_attribute('value')
                    ## color_value 안에 [품절]이란 단어 유무에 따른 color_name 선정
                    if "[품절]" in color_value:
                        color_name = color_value.replace(" [품절]", "")
                    else:
                        color_name = color_value
                    
                    base_code = base_name
                    
                    select = Select(text_info.find_element('xpath', '//select[@option_title="색상"]'))
                    try:
                        select.select_by_visible_text(color_name)
                    except NoSuchElementException:
                        try:
                            color_text = color_name + " [품절]"
                            select.select_by_visible_text(color_text)
                        except:
                            continue
                    except ElementNotInteractableException:
                        continue
                        
                    size_elements = text_info.find_elements('xpath', '//select[@option_title="사이즈"]/option')[2:]
                    size = "M"
                    is_available = True

                    for element in size_elements:
                        size = element.text.replace(" [품절]","")
                        if "[품절]" in element.text:
                            is_available = False
                        else:
                            is_available = True
                    
                    append_data(base_dict, cat, base_name, base_code, baseURL, color_name, base_gender, price, base_price, color_fabric, imageURL, size, is_available)
                
            
            
            
        ## option_title이 '컬러'로 되어 있는 경우
        elif len(text_info.find_elements('xpath', '//select[@option_title="컬러"]')) != 0:
            
            ## base_name에 옷의 색 정보가 포함되어있는 경우
            if len(text_info.find_elements('xpath', '//select[@option_title="컬러"]/option')) == 3:
                         
                color_name = base_name.split(" ")[-1]
                base_code = base_name.replace(color_name, "")[:-1]
            
                select = Select(text_info.find_element('xpath', '//select[@option_title="컬러"]'))
                try:
                    select.select_by_visible_text(color_name)
                except NoSuchElementException:
                    try:
                        color_text = color_name + " [품절]"
                        select.select_by_visible_text(color_text)
                    except:
                        continue
                except ElementNotInteractableException:
                    continue
                    
                size_elements = text_info.find_elements('xpath', '//select[@option_title="사이즈"]/option')[2:]
                size = "M"
                is_available = True

                for element in size_elements:
                    size = element.text.replace(" [품절]","")
                    if "[품절]" in element.text:
                        is_available = False
                    else:
                        is_available = True
                append_data(base_dict, cat, base_name, base_code, baseURL, color_name, base_gender, price, base_price, color_fabric, imageURL, size, is_available)
            
            
            ## base_name에 옷의 색 정보가 포함되어 있지 않은 경우
            else:
                option_cnt = len(text_info.find_elements('xpath', '//select[@option_title="컬러"]/option'))
                
                ## 각 색깔 별로 따로 정보를 추출 및 저장해야 함
                for i in range(2,option_cnt):
                    color_value = text_info.find_elements('xpath', '//select[@option_title="컬러"]/option')[i].get_attribute('value')
                    
                    ## color_value 안에 [품절]이란 단어 유무에 따른 color_name 선정
                    if "[품절]" in color_value:
                        color_name = color_value.replace(" [품절]", "")
                    else:
                        color_name = color_value
                    
                    base_code = base_name
                    
                    select = Select(text_info.find_element('xpath', '//select[@option_title="컬러"]'))
                    try:
                        select.select_by_visible_text(color_name)
                    except NoSuchElementException:
                        try:
                            color_text = color_name + " [품절]"
                            select.select_by_visible_text(color_text)
                        except:
                            continue
                    except ElementNotInteractableException:
                        continue
                        
                    size_elements = text_info.find_elements('xpath', '//select[@option_title="사이즈"]/option')[2:]
                    size = "M"
                    is_available = True

                    for element in size_elements:
                        size = element.text.replace(" [품절]","")
                        if "[품절]" in element.text:
                            is_available = False
                        else:
                            is_available = True
                            
                    append_data(base_dict, cat, base_name, base_code, baseURL, color_name, base_gender, price, base_price, color_fabric, imageURL, size, is_available)
            
            
            
        ## option_title이 없는 경우 (컬러 선택할 수 없는 경우)
        else:
            continue # 너무 예외적인 케이스라 pass
            
    
    print("len: ", len(base_dict))
    
    return base_dict

In [9]:
import pandas as pd

In [10]:
urls = [
        ('https://whoisnerdy.com/product/list.html?cate_no=30', 15), #상의
        ('https://whoisnerdy.com/product/list.html?cate_no=61', 5), #자켓
        ('https://whoisnerdy.com/product/list.html?cate_no=31', 7), #하의
         ]

product_paths = find_product_paths(urls)
print(len(product_paths))

784


In [11]:
all_color_paths = find_all_color_paths(product_paths)
print(len(all_color_paths))

1540


In [12]:
## remove duplicates
all_color_paths = list(set(all_color_paths))
print(len(all_color_paths))

1127


In [13]:
## remove 품절 상품
for path in all_color_paths:
    if type(path) is not str:
        all_color_paths.remove(path)
print(len(all_color_paths))

1126


In [26]:
# a = ["https://whoisnerdy.com/product/detail.html?product_no=5050&cate_no=30&display_group=1#none"]

# crawl_pages(a)

len:  12


{'baseCat': [],
 'baseName': [],
 'baseCode': [],
 'baseUrl': [],
 'colorName': [],
 'baseGender': [],
 'price': [],
 'basePrice': [],
 'colorFabric': [],
 'imageUrl': [],
 'size': [],
 'isAvailable': []}

In [29]:
dict1 = crawl_pages(all_color_paths[:500])
df1 = pd.DataFrame.from_dict(dict1)

print(df1)

len:  12
    baseCat                baseName             baseCode  \
0    bottom            에센셜 조거 팬츠 블랙            에센셜 조거 팬츠   
1       top  우먼스 컬러 블락 윈드브레이커 세트 블랙  우먼스 컬러 블락 윈드브레이커 세트   
2       top   레이어드 커시브 로고 풀오버 후드 블랙   레이어드 커시브 로고 풀오버 후드   
3       top          타이다잉 크롭 가디건 코랄          타이다잉 크롭 가디건   
4    bottom                벨벳 트랙 팬츠             벨벳 트랙 팬츠   
..      ...                     ...                  ...   
425  bottom        커브드 와이드 트랙 팬츠 블랙        커브드 와이드 트랙 팬츠   
426     top    널디코니 크롭 반팔 티셔츠 라이트퍼플       널디코니 크롭 반팔 티셔츠   
427     top          사이드 라인 크롭 탑 블루          사이드 라인 크롭 탑   
428     top        피케 폴로 반팔 티셔츠 화이트         피케 폴로 반팔 티셔츠   
429     top            레이싱 트랙 팬츠 블랙            레이싱 트랙 팬츠   

                                               baseUrl colorName  baseGender  \
0    https://whoisnerdy.com/product/detail.html?pro...        블랙           0   
1    https://whoisnerdy.com/product/detail.html?pro...        블랙           2   
2    https://whoisnerdy.com/pr

In [32]:
dict2 = crawl_pages(all_color_paths[500:1000])
df2 = pd.DataFrame.from_dict(dict2)

print(df2)

len:  12
    baseCat                  baseName               baseCode  \
0       top      엔와이 사이드 블락 스웻팬츠 다크퍼플        엔와이 사이드 블락 스웻팬츠   
1       top        우먼스 수플레 크롭 스웻셔츠 크림        우먼스 수플레 크롭 스웻셔츠   
2       top          빈티지 로고 풀오버 후디 차콜          빈티지 로고 풀오버 후디   
3       top          머니 토끼 반팔 티셔츠 화이트           머니 토끼 반팔 티셔츠   
4       top        NY 반팔 윈드브레이커 세트 크림        NY 반팔 윈드브레이커 세트   
..      ...                       ...                    ...   
403     top  널디 X 몬타나 피케 폴로 반팔 티셔츠 크림  널디 X 몬타나 피케 폴로 반팔 티셔츠   
404     top         유니 컬러 블락 우븐 세트 핑크         유니 컬러 블락 우븐 세트   
405     top           우먼스 크롭 후디 집업 블랙           우먼스 크롭 후디 집업   
406     top   디즈니 X 널디 소시지 반팔 티셔츠 화이트    디즈니 X 널디 소시지 반팔 티셔츠   
407  bottom               에센셜 스웻팬츠 블랙               에센셜 스웻팬츠   

                                               baseUrl colorName  baseGender  \
0    https://whoisnerdy.com/product/detail.html?pro...      다크퍼플           0   
1    https://whoisnerdy.com/product/detail.html?pro...        

In [33]:
dict3 = crawl_pages(all_color_paths[1000:])
df3 = pd.DataFrame.from_dict(dict3)

print(df3)

len:  12
    baseCat                         baseName                      baseCode  \
0       top  널디 X 몬타나 우먼스 피케 폴로 크롭 반팔 티셔츠 크림  널디 X 몬타나 우먼스 피케 폴로 크롭 반팔 티셔츠   
1       top             다이애그널 라인 스웻셔츠 멜란지그레이                 다이애그널 라인 스웻셔츠   
2       top               베이직 빅 로고 반팔 티셔츠 블루               베이직 빅 로고 반팔 티셔츠   
3    bottom                   엔와이 우븐 트랙 하프팬츠                엔와이 우븐 트랙 하프팬츠   
4    bottom                   엔와이 우븐 트랙 하프팬츠                엔와이 우븐 트랙 하프팬츠   
..      ...                              ...                           ...   
103     top                  빅 로고 스웻셔츠 라이트퍼플                     빅 로고 스웻셔츠   
104     top                빅 로고 풀오버 후디 라이트퍼플                   빅 로고 풀오버 후디   
105  bottom                 NY 바이컬러 트랙 팬츠 블랙                 NY 바이컬러 트랙 팬츠   
106     top               우먼스 부클 로고 스웻셔츠 네이비                우먼스 부클 로고 스웻셔츠   
107     top                  체커보드 테잎 스웻셔츠 크림                  체커보드 테잎 스웻셔츠   

                                               baseUrl

In [34]:
df_final = pd.concat([df1, df2, df3])

In [35]:
df_final.to_csv('result.csv',index=False)